## single query on document

In [99]:
# !pip install python-dotenv

In [100]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [15]:
import os
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  #

In [103]:
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [47]:
# @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))

In [48]:
documents = SimpleDirectoryReader('../data').load_data()

In [49]:
documents

[Document(text='CBC (Complete Blood Count), Blood\nPARAMETER\nRESULTS\nBIOLOGICAL REF RANGE\nMETHOD\nRBC PARAMETERS\nHaemoglobin \n11.8 \n12.0-15.0 g/dL\nSpectrophotometric\nRBC \n4.61 \n3.8-4.8 mil/cmm\nElect. Impedance\nPCV \n37.6 \n36-46 %\nMeasured\nMCV \n81.5 \n80-100 fl\nCalculated\nMCH \n25.5 \n27-32 pg\nCalculated\nMCHC \n31.3 \n31.5-34.5 g/dL\nCalculated\nRDW \n12.5 \n11.6-14.0 %\nCalculated\nWBC PARAMETERS\nWBC Total Count \n17800 \n4000-10000 /cmm\nElect. Impedance\nWBC DIFFERENTIAL AND ABSOLUTE COUNTS\nLymphocytes \n12.9 \n20-40 %\nAbsolute Lymphocytes \n2296.2 \n1000-3000 /cmm\nCalculated\nMonocytes \n11.0 \n2-10 %\nAbsolute Monocytes \n1958.0 \n200-1000 /cmm\nCalculated\nNeutrophils \n74.5 \n40-80 %\nAbsolute Neutrophils \n13261.0 \n2000-7000 /cmm\nCalculated\nEosinophils \n1.2 \n1-6 %\nAbsolute Eosinophils \n213.6 \n20-500 /cmm\nCalculated\nBasophils \n0.4 \n0.1-2 %\nAbsolute Basophils \n71.2 \n20-100 /cmm\nCalculated\nImmature Leukocytes \n-  \nWBC Differential Count by

In [212]:
type(documents)

list

In [215]:
docs = []
for doc in documents:
    docs.append(doc.text)
#documents[0].text

In [220]:
doc_text = ' '.join(docs).replace('\n', ' ')
doc_text

"CBC (Complete Blood Count), Blood PARAMETER RESULTS BIOLOGICAL REF RANGE METHOD RBC PARAMETERS Haemoglobin  11.8  12.0-15.0 g/dL Spectrophotometric RBC  4.61  3.8-4.8 mil/cmm Elect. Impedance PCV  37.6  36-46 % Measured MCV  81.5  80-100 fl Calculated MCH  25.5  27-32 pg Calculated MCHC  31.3  31.5-34.5 g/dL Calculated RDW  12.5  11.6-14.0 % Calculated WBC PARAMETERS WBC Total Count  17800  4000-10000 /cmm Elect. Impedance WBC DIFFERENTIAL AND ABSOLUTE COUNTS Lymphocytes  12.9  20-40 % Absolute Lymphocytes  2296.2  1000-3000 /cmm Calculated Monocytes  11.0  2-10 % Absolute Monocytes  1958.0  200-1000 /cmm Calculated Neutrophils  74.5  40-80 % Absolute Neutrophils  13261.0  2000-7000 /cmm Calculated Eosinophils  1.2  1-6 % Absolute Eosinophils  213.6  20-500 /cmm Calculated Basophils  0.4  0.1-2 % Absolute Basophils  71.2  20-100 /cmm Calculated Immature Leukocytes  -   WBC Differential Count by Absorbance & Impedance method/Microscopy. PLATELET PARAMETERS Platelet Count  336000  15000

In [50]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(1))
def indexGPTVectorStoreFromDocuments(documents):
    index = GPTVectorStoreIndex.from_documents(documents)
    return index

In [51]:
index = indexGPTVectorStoreFromDocuments(documents)

In [52]:
query_engine = index.as_query_engine()

In [54]:
response = query_engine.query("What is the name of the patient?")

In [55]:
type(response)

llama_index.response.schema.Response

In [62]:
print(response)


MRS.RESHMA HEMDEV OPD


In [63]:
# To persist to disk (under ./storage):
index.storage_context.persist()

In [65]:
!ls storage

docstore.json     index_store.json  vector_store.json


In [68]:
!cat storage/index_store.json

{"index_store/data": {"b28e0ea9-43f9-4cd9-8e07-7e9ee2559021": {"__type__": "vector_store", "__data__": "{\"index_id\": \"b28e0ea9-43f9-4cd9-8e07-7e9ee2559021\", \"summary\": null, \"nodes_dict\": {\"fee5b18b-4363-40b7-8f4a-7248c9af7923\": \"fee5b18b-4363-40b7-8f4a-7248c9af7923\", \"446dcb5e-da5c-4e39-a0b4-66de22f5d7a7\": \"446dcb5e-da5c-4e39-a0b4-66de22f5d7a7\", \"e8e9216b-4d7e-4db8-a3d7-bdb21bba8706\": \"e8e9216b-4d7e-4db8-a3d7-bdb21bba8706\", \"44ecbb29-c41f-4744-a1e6-b69fd235e805\": \"44ecbb29-c41f-4744-a1e6-b69fd235e805\"}, \"doc_id_dict\": {}, \"embeddings_dict\": {}}"}}}

In [ ]:
# code to reload from the disk
from llama_index import StorageContext, load_index_from_storage

# To persist to disk (under ./storage):
index.storage_context.persist()

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir='./storage')
# load index
index = load_index_from_storage(storage_context)

## Chat Engines Condense Question Mode

In [208]:
chat_engine = index.as_chat_engine(verbose=False)

In [137]:
# prompt = f"Give me a summary of the medical report in laymen terms? Any person that does not have much understanding\
#             about the medical term should be able to understand."
# response = chat_engine.chat(prompt)

In [138]:
# print(response)

In [ ]:
index.as_chat_engine?

In [209]:
prompt_2 = f"""
You are an AI assistant who can understand medical reports and summarize them in layman's terms. \
Summarize the medical report in layman's terms use minimum 120 words to describe the summary. Only use \
most relevant information from the document that is useful to the person whose report is this. The summary \
should contain very simple and easy to understand terms so that even a person with no knowledge of medical science \
can understand the summary. \
Also, extract the parameter readings from the biomedical report. \
Format your response in the following format: \
Summary: <Summary of the report in layman's terms>. \
------
Extracted Data: <List of JSON objects>. \
The list of JSON objects should have "Parameter," "Result," "Biological Reference Range," and "Method" as the keys. \
If the information isn't present, use "unknown" as the value.

An example of the report will be like this: \
Summary: "This is the summary of the report" \
Extracted Data: \
[{{"Parameter": "Haemoglobin","Result": "11.8","Biological Reference Range": "12.0-15.0 g/dL","Method": "Spectrophotometric"}}]
"""

In [210]:
# prompt_2 = f"""
# You are an ai assistant who can understand the medical reports and summarise in laymen terms. \
# Summarise the medical report in laymen terms, even any person that does not know anything about medical science \
# should be able to understand the summary. \
# Also extract the parameter readings from the biomedical report. \
# Format your response in the follwing format:- \
# Summary:- <Summary of the report in laymen term>.
# Extracted Data:- <List of Json object>. \
# The list of JSON object with "Parameter", "Result", "Biological Reference Range" and "Method" as the keys. \
# If the information isn't present, use "unknown" as the value.

# An example of the report will be like this:- \
# Summary:- "This is the summary of the report" \
# Extracted Data:- \
# [{"Parameter": "Haemoglobin","Result": "11.8","Biological Reference Range": "12.0-15.0 g/dL","Method": "Spectrophotometric"}]
# """

response_2 = chat_engine.chat(prompt_2)
print(response_2)


Summary: Mrs. Reshma Hemdev OPD, a 55-year-old female, underwent a complete blood count (CBC) test, a malarial parasite (MP) test, and a C-reactive protein (CRP) quantitative test. The CBC test revealed a haemoglobin level of 11.8 g/dL, a red blood cell count of 4.61 mil/cmm, a packed cell volume of 37.6%, a mean corpuscular volume of 81.5 fl, a mean corpuscular haemoglobin of 25.5 pg, a mean corpuscular haemoglobin concentration of 31.3 g/dL, a red cell distribution width of 12.5%, a white blood cell count of 17800/cmm, a lymphocyte count of 12.9%, an absolute lymphocyte count of 2296.2/cmm, a monocyte count of 11.0%, an absolute monocyte count of 1958.0/cmm, a neutrophil count of 74.5%, an absolute neutrophil count of 13261.0/cmm, an eosinophil count of 1.2%, an absolute eosinophil count of 213.6/cmm, a basophil count of 0.4%, and an absolute basophil count of 71.2/cmm. The MP test was negative. The CRP quantitative test revealed a result of 219.9 mg/L.

Extracted Data: [{"Parameter

In [211]:
# 'Extract the out of range reading of the patient, the abnormal reading\
#     is the reading whose value is not in the range of biological reference, \
#     before returning the result validate the result by your self. For example if the result is 11 and the refrence \
#     range is between 10-12 then the value is in range but if the value is 13 then it is out of range.Return the result in json format with keys as parameter, \
# parameter value and biological reference range ?'

# delimeter = "```"
prompt = f"""
Extract the parameter reading from the biomedical report. \
Format your response as a list of JSON object with \
"Parameter", "Result", "Biological Reference Range" and "Method" as the keys. \
If the information isn't present, use "unknown" as the value.
"""

response = chat_engine.chat(prompt)
print(response)

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).



The medical report is for Mrs. Reshma Hemdev OPD, a 55-year-old female, with CID 2117740429. The report includes a Complete Blood Count (CBC), Malarial Parasite (MP) and C-Reactive Protein (CRP) Quantitative Test. 

The CBC test results show that Mrs. Hemdev's haemoglobin level is 11.8 g/dL, which is slightly lower than the normal range of 12.0-15.0 g/dL. Her red blood cell count is 4.61 mil/cmm, which is within the normal range of 3.8-4.8 mil/cmm. Her packed cell volume (PCV) is 37.6%, which is within the normal range of 36-46%. Her mean corpuscular volume (MCV) is 81.5 fl, which is within the normal range of 80-100 fl. Her mean corpuscular haemoglobin (MCH) is 25.5 pg, which is lower than the normal range of 27-32 pg. Her mean corpuscular haemoglobin concentration (MCHC) is 31.3 g/dL, which is within the normal range of 31.5-34.5 g/dL. Her red cell distribution width (RDW) is 12.5%, which is within the normal range of 11.6-14.0%.

The WBC test results show that Mrs. Hemdev's total w

In [167]:
reports = json.loads(response.response.split('\n\n')[-1])
print(reports)

[{'Parameter': 'Haemoglobin', 'Result': '11.8', 'Biological Reference Range': '12.0-15.0 g/dL', 'Method': 'Spectrophotometric'}, {'Parameter': 'RBC', 'Result': '4.61', 'Biological Reference Range': '3.8-4.8 mil/cmm', 'Method': 'Elect. Impedance'}, {'Parameter': 'PCV', 'Result': '37.6', 'Biological Reference Range': '36-46 %', 'Method': 'Measured'}, {'Parameter': 'MCV', 'Result': '81.5', 'Biological Reference Range': '80-100 fl', 'Method': 'Calculated'}, {'Parameter': 'MCH', 'Result': '25.5', 'Biological Reference Range': '27-32 pg', 'Method': 'Calculated'}, {'Parameter': 'MCHC', 'Result': '31.3', 'Biological Reference Range': '31.5-34.5 g/dL', 'Method': 'Calculated'}, {'Parameter': 'RDW', 'Result': '12.5', 'Biological Reference Range': '11.6-14.0 %', 'Method': 'Calculated'}, {'Parameter': 'WBC Total Count', 'Result': '17800', 'Biological Reference Range': '4000-10000 /cmm', 'Method': 'Elect. Impedance'}, {'Parameter': 'Lymphocytes', 'Result': '12.9', 'Biological Reference Range': '20-4

In [153]:
float("20-100 /cmm")

ValueError: could not convert string to float: '20-100 /cmm'

In [170]:
def validate_value_in_range(record):
    parameter_value = float(record['Result'])
    biological_range = record['Biological Reference Range'].split(' ')[0]
    biological_low_range, biological_high_range  = [float(val) for val in biological_range.split('-')]
    if parameter_value < biological_low_range:
        record['variation'] = str(parameter_value - biological_low_range)
    elif parameter_value > biological_high_range:
        record['variation'] = str(parameter_value - biological_high_range)
    return record

In [171]:
import json
def get_relevant_report(reports):
    # resports data format is expected to be a list of json object
    abnormal_data_numeric = []
    string_data = []
    for record in reports:
        if record["Result"] != "unknown":
            try:
                new_record = validate_value_in_range(record)
                abnormal_data_numeric.append(new_record)
            except Exception as e:
                # if here means the vlaue is string type
                string_data.append(record)
    new_report = {
        "numeric": abnormal_data_numeric,
        "string": string_data
    }
    return new_report

In [172]:
get_relevant_report(reports)

{'numeric': [{'Parameter': 'Haemoglobin',
   'Result': '11.8',
   'Biological Reference Range': '12.0-15.0 g/dL',
   'Method': 'Spectrophotometric',
   'variation': '-0.1999999999999993'},
  {'Parameter': 'RBC',
   'Result': '4.61',
   'Biological Reference Range': '3.8-4.8 mil/cmm',
   'Method': 'Elect. Impedance'},
  {'Parameter': 'PCV',
   'Result': '37.6',
   'Biological Reference Range': '36-46 %',
   'Method': 'Measured'},
  {'Parameter': 'MCV',
   'Result': '81.5',
   'Biological Reference Range': '80-100 fl',
   'Method': 'Calculated'},
  {'Parameter': 'MCH',
   'Result': '25.5',
   'Biological Reference Range': '27-32 pg',
   'Method': 'Calculated',
   'variation': '-1.5'},
  {'Parameter': 'MCHC',
   'Result': '31.3',
   'Biological Reference Range': '31.5-34.5 g/dL',
   'Method': 'Calculated',
   'variation': '-0.1999999999999993'},
  {'Parameter': 'RDW',
   'Result': '12.5',
   'Biological Reference Range': '11.6-14.0 %',
   'Method': 'Calculated'},
  {'Parameter': 'WBC Tota

Response(response='\nYes, you can provide a list of JSON objects containing the "Parameter", "Result", "Biological Reference Range" and "Method" from the biomedical report, with "unknown" as the value if the information isn\'t present. An example of such a list is provided below:\n\n[\n  {\n    "Parameter": "Haemoglobin",\n    "Result": "11.8",\n    "Biological Reference Range": "12.0-15.0 g/dL",\n    "Method": "Spectrophotometric"\n  },\n  {\n    "Parameter": "RBC",\n    "Result": "4.61",\n    "Biological Reference Range": "3.8-4.8 mil/cmm",\n    "Method": "Elect. Impedance"\n  },\n  {\n    "Parameter": "PCV",\n    "Result": "37.6",\n    "Biological Reference Range": "36-46 %",\n    "Method": "Measured"\n  },\n  {\n    "Parameter": "MCV",\n    "Result": "81.5",\n    "Biological Reference Range": "80-100 fl",\n    "Method": "Calculated"\n  },\n  {\n    "Parameter": "MCH",\n    "Result": "25.5",\n    "Biological Reference Range": "27-32 pg",\n    "Method": "Calculated"\n  },\n  {\n    "

In [201]:
text = "Does the RBC count normal?"
lang = "Hindi"

prompt = f"{text}. Return the result in {lang} language."

response = chat_engine.chat(prompt)
print(response)


हाँ, रेड ब्लड सेल का परिणाम नॉर्मल रेंज में है।


In [81]:
response = chat_engine.chat('Can you return the complete blood count report as HTML table only?')
print(response)

In [79]:
response.response

'\nYes, you can provide the complete blood count report for Mrs. Reshma Hemdev as an HTML table, including the RBC count which is 4.61 mil/cmm and is within the normal range of 3.8-4.8 mil/cmm.\n\n<table>\n  <tr>\n    <th>PARAMETER</th>\n    <th>RESULTS</th>\n    <th>BIOLOGICAL REF RANGE</th>\n    <th>METHOD</th>\n  </tr>\n  <tr>\n    <td>RBC PARAMETERS</td>\n    <td></td>\n    <td></td>\n    <td></td>\n  </tr>\n  <tr>\n    <td>Haemoglobin</td>\n    <td>11.8</td>\n    <td>12.0-15.0 g/dL</td>\n    <td>Spectrophotometric</td>\n  </tr>\n  <tr>\n    <td>RBC</td>\n    <td>4.61</td>\n    <td>3.8-4.8 mil/cmm</td>\n    <td>Elect. Impedance</td>\n  </tr>\n  <tr>\n    <td>PCV</td>\n    <td>37.6</td>\n    <td>36-46 %</td>\n    <td>Measured</td>\n  </tr>\n  <tr>\n    <td>MCV</td>\n    <td>81.5</td>\n    <td>80-100 fl</td>\n    <td>Calculated</td>\n  </tr>\n  <tr>\n    <td>MCH</td>\n    <td>25.5</td>\n    <td>27-32 pg</td>\n    <td>Calculated</td>\n  </tr>\n  <tr>\n    <td>MCHC</td>\n    <td>31.3</

In [80]:
from IPython.core.display import display, HTML
display(HTML(response.response))

<ipython-input-80-03d1f0c8f367>:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


PARAMETER,RESULTS,BIOLOGICAL REF RANGE,METHOD
RBC PARAMETERS,,,
Haemoglobin,11.8,12.0-15.0 g/dL,Spectrophotometric
RBC,4.61,3.8-4.8 mil/cmm,Elect. Impedance
PCV,37.6,36-46 %,Measured
MCV,81.5,80-100 fl,Calculated
MCH,25.5,27-32 pg,Calculated
MCHC,31.3,31.5-34.5 g/dL,Calculated
RDW,12.5,11.6-14.0 %,Calculated
WBC PARAMETERS,,,
WBC Total Count,17800,4000-10000 /cmm,Elect. Impedance


## summarization

In [83]:
from llama_index import GPTListIndex
index = GPTListIndex.from_documents(documents)

query_engine = index.as_query_engine(
    response_mode="tree_summarize"
)

In [104]:
# prompt = f"<summarization_query>"
prompt = f"Give me a summary of the medical report? Return the critial points as html list. Make sure the output is only in HTML format."
response = query_engine.query(prompt)
print(response)


<ul>
  <li>Complete Blood Count (CBC) results: Haemoglobin 11.8 g/dL, RBC 4.61 mil/cmm, PCV 37.6 %, MCV 81.5 fl, MCH 25.5 pg, MCHC 31.3 g/dL, RDW 12.5 %</li>
  <li>WBC Total Count 17800 /cmm, Lymphocytes 12.9 %, Absolute Lymphocytes 2296.2 /cmm, Monocytes 11.0 %, Absolute Monocytes 1958.0 /cmm, Neutrophils 74.5 %, Absolute Neutrophils 13261.0 /cmm, Eosinophils 1.2 %, Absolute Eosinophils 213.6 /cmm, Basophils 0.4 %, Absolute Basophils 71.2 /cmm, Immature Leukocytes -</li>
  <li>Platelet Count 336000 /cmm, MPV 8.5 fl, PDW 11.8 %</li>
  <li>RBC Morphology: Hypochromia Mild</li>
  <li>Malarial Parasite (MP) test result: Negative</li>
  <li>CRP-Quantitative Test result: 219.9 mg/l</li>
  <li>CRP test is not diagnostic of any condition, but it can be used together with signs and symptoms and other tests to evaluate an individual for an acute or chronic inflammatory condition.</li>
  <li>Reflex Tests: Complement, Procalcitonin</li>
</ul>


In [107]:
prompt = f"Give me a summary of the medical report in laymen terms? Any person that does not have much understanding\
            about the medical term should be able to understand. Return the response in Hindi Language"
response = query_engine.query(prompt)
print(response)


मूलत: यह रिपोर्ट एक 55 वर्षीय महिला के लिए की गई थी। रिपोर्ट में रक्त की पूरी माहिती शामिल है, जैसे कि हेमोग्लोबिन, रेड ब्लड केल्लेस, प्लेटेट की गणना, मलेरिया पैरासाइट और CRP क्वांटिटेटिव टेस्ट। रिपोर्ट में दिखाया गया है कि रक्त में न्यूट्रोफिल ल्यूकोसाइटिस है और CRP की स्तर उच्च है। इसका मतलब है कि शरीर में एक तीव्र इन्फ्लेमेशन हो रहा है।


In [106]:
from IPython.display import display, Markdown, Latex, HTML, JSON

html_text = """
<ul>
  <li>Complete Blood Count (CBC) results: Haemoglobin 11.8 g/dL, RBC 4.61 mil/cmm, PCV 37.6 %, MCV 81.5 fl, MCH 25.5 pg, MCHC 31.3 g/dL, RDW 12.5 %</li>
  <li>WBC Total Count 17800 /cmm, Lymphocytes 12.9 %, Absolute Lymphocytes 2296.2 /cmm, Monocytes 11.0 %, Absolute Monocytes 1958.0 /cmm, Neutrophils 74.5 %, Absolute Neutrophils 13261.0 /cmm, Eosinophils 1.2 %, Absolute Eosinophils 213.6 /cmm, Basophils 0.4 %, Absolute Basophils 71.2 /cmm, Immature Leukocytes -</li>
  <li>Platelet Count 336000 /cmm, MPV 8.5 fl, PDW 11.8 %</li>
  <li>RBC Morphology: Hypochromia Mild</li>
  <li>Malarial Parasite (MP) test result: Negative</li>
  <li>CRP-Quantitative Test result: 219.9 mg/l</li>
  <li>CRP test is not diagnostic of any condition, but it can be used together with signs and symptoms and other tests to evaluate an individual for an acute or chronic inflammatory condition.</li>
  <li>Reflex Tests: Complement, Procalcitonin</li>
</ul>
"""
display(HTML(html_text))

## Chat Engine - ReAct Mode

## openai api test

In [101]:
import os
import openai

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [97]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [98]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Classify each query into a primary category \
and a secondary category. 
Provide your output in json format with the \
keys: primary and secondary.

Primary categories: Billing, Technical Support, \
Account Management, or General Inquiry.

Billing secondary categories:
Unsubscribe or upgrade
Add a payment method
Explanation for charge
Dispute a charge

Technical Support secondary categories:
General troubleshooting
Device compatibility
Software updates

Account Management secondary categories:
Password reset
Update personal information
Close account
Account security

General Inquiry secondary categories:
Product information
Pricing
Feedback
Speak to a human

"""
user_message = f"""\
I want you to delete my profile and all of my user data"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

{
  "primary": "Account Management",
  "secondary": "Close account"
}
